## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8281462 , -0.02252063, -0.5600595 ,  0.64759785],
        [ 0.17067207,  0.96187705,  0.21369046, -0.16164917],
        [ 0.5338959 , -0.27255347,  0.8004185 ,  0.09115803],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
generate table - Geometry
generate table - Object
generate table - Handle
generate table - Binder
generate table - Marker
generate table - MarkerGroup
generate table - Geometry
generate table - Object


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

Please create a subscriber to the marker


In [8]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))

In [9]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [10]:
 gscene.NAME_DICT["floor"].dims = (2,1,0.01)

In [11]:
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.1)

## add sweep face
* Put "track" marker as shown in the figure, on the edge of the whiteboard
* After detecting the track, remove the marker to make it possible to sweep the surface

In [12]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [13]:
gscene.update_markers_all()

## add wp

In [14]:
TOOL_DIM = [0.06, 0.10]
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    line = gscene.create_safe(GEOTYPE.BOX, "trc{}".format(i_trc+1), "base_link", (TRACK_DIM[0]*2, TRACK_DIM[0]*2, 1e-6), 
                               (0,WP_REF_A[1]+TRACK_STEP*i_trc,0), rpy=(np.pi/2,0,0), 
                               color=(0.8,0.2,0.2,0.2), display=False, fixed=True, collision=False, parent="track_face")
    line_vis = gscene.create_safe(GEOTYPE.BOX, "trc{}vis".format(i_trc+1), "base_link", (TRACK_DIM[0], 0.01, 1e-6), 
                               (0,WP_REF_A[1]+TRACK_STEP*i_trc,0), rpy=(np.pi/2,0,0), 
                               color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, line))

In [15]:
gscene.update_markers_all()

## add indy tool

In [16]:
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.07,0.07, 0.04), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts_col", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.11,0.11, 0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.03,0.03, 0.030), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole_col", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.07,0.07, 0.030), rpy=(0,0,0), color=(0.0,0.8,0.0,0.2),
                            collision=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="indy0_brushbase", link_name="indy0_tcp",
                            center=(0,0,0.0775), dims=(0.06, 0.14, 0.015), rpy=(0,0,np.pi/4), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp",
                            center=(0,0,0.095), dims=(0.05,0.13, 0.015), rpy=(0,0,np.pi/4), color=(1.0,1.0,0.94,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp",
                            center=(0,0,0.080), dims=(0.08,0.18, 0.02), rpy=(0,0,np.pi/4), color=(0.0,0.8,0.0,0.5),
                            collision=True)

## add box

In [24]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [25]:
# give clearance from the track surface
s_builder.give_clearance(track, gtem_dict.values())

In [26]:
gscene.update_markers_all()

## Register binders

In [27]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, SweepFramer, FixtureSlot

In [28]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,0.00), 
                     rpy=(0,np.pi,0))

## add objects

In [29]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [30]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [31]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [32]:
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, tline = track_tem
    sweep1 = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                 action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])},
                                geometry_vertical = tline)

### planners

In [33]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [34]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [35]:
mplan.motion_filters = checkers_all

In [36]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [37]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('track_face', 'track_face', 0, 0, 0)


In [38]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [39]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [40]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan

In [41]:
mplan.reset_log(False)
gtimer.reset()
# goal_nodes = [("track_face","track_face")+(2,0,0)]
goal_nodes = [("track_face","track_face")+tuple([2 for _ in range(TRACK_NUM)])]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=200, multiprocess=True, timeout=10, timeout_constrained=50)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
try transition motion
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
try: 0 - (

try: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 9 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
result: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0) = fail
transition motion tried: True
try: 0 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 0 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 6 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 10 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
try: 0 - ('track_face', 'tra

branching: 18->19 (1.01/200.0 s, steps/err: 32(88.0010128021 ms)/4.97330864409e-16)
result: 13 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
result: 19 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0) = fail
result: 17 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'track_face', 1, 0, 0) = success
try: 1 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0)
branching: 17->20 (1.03/200.0 s, steps/err: 14(132.931947708 ms)/0.00165540422309)
try: 20 - ('track_face', 'track_face', 1, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 13 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 3 - ('grip1', 'track_face', 0, 0, 0)->('track_face', 'track_face', 0, 0, 0) = fail
try: 13 - ('track_face', 'track_face', 0, 0, 0)->('grip1', 'track_face', 0, 0, 0)
result: 13 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = fail
try constrained motion
transition motio

branching: 11->27 (1.38/200.0 s, steps/err: 21(155.035972595 ms)/0.0014410602025)
transition motion tried: True
try joint motion
result: 24 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = success
try: 28 - ('grip1', 'track_face', 1, 0, 0)->('grip1', 'track_face', 2, 0, 0)
branching: 24->28 (1.41/200.0 s, steps/err: 14(124.19295311 ms)/0.00170421199421)
try: 27 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0)
try: 29 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0)
result: 11 - ('track_face', 'track_face', 0, 0, 0)->('track_face', 'grip1', 0, 0, 0) = success
try constrained motion
try: 29 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'grip1', 1, 0, 0)
joint motion tried: True
branching: 11->29 (1.45/200.0 s, steps/err: 23(212.709903717 ms)/0.00087961322665)
try joint motion
result: 27 - ('grip1', 'track_face', 0, 0, 0)->('grip1', 'track_face', 1, 0, 0) = fail
try: 30 - ('track_face', 'grip1', 0, 0, 0)->('track_face', 'g

try: 36 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
result: 35 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 32 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0) = success
try: 36 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 32 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0)
try: 38 - ('grip1', 'track_face', 2, 1, 0)->('grip1', 'track_face', 2, 2, 0)
result: 32 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
branching: 32->38 (1.8/200.0 s, steps/err: 8(206.230163574 ms)/0.000777234044328)
result: 32 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 36 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
result: 32 - ('grip1', 'track_face', 2, 0, 0)->('track_face', 'track_face', 2, 0, 0) = fail
try: 35 - ('grip1', 'track_face', 2, 0, 0)->(

...

result: 63 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0) = fail
try: 71 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
try: 70 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0)
try joint motion
result: 25 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0) = fail
try: 42 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
try: 55 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
try: 45 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 2, 0, 0)
try: 71 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0)
try: 45 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0)
transition motion tried: True
try transition motion
joint motion tried: True
result: 42 - ('track_face', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0) = fail
result: 45 - ('track_face', 'track_face', 2, 0, 0)->('track_face', 'gri

try: 75 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try: 78 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try constrained motion
joint motion tried: True
result: 77 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 0, 0) = success
result: 78 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
result: 75 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = fail
try: 81 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0)
branching: 77->81 (3.52/200.0 s, steps/err: 15(129.081964493 ms)/5.28712635324e-16)
try: 75 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
try transition motion
transition motion tried: True
try: 78 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0)
transition motion tried: True
result: 79 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0) = success
result: 78 - ('tr

joint motion tried: True
try: 91 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1)
transition motion tried: True
result: 87 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 0) = success
try joint motion
result: 82 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
try: 92 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1)
result: 89 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = success
result: 91 - ('track_face', 'grip1', 2, 2, 0)->('track_face', 'grip1', 2, 2, 1) = fail
try: 93 - ('track_face', 'track_face', 2, 2, 1)->('track_face', 'track_face', 2, 2, 2)
branching: 87->92 (4.24/200.0 s, steps/err: 38(286.074876785 ms)/9.31339010618e-16)
branching: 89->93 (4.26/200.0 s, steps/err: 27(170.259952545 ms)/0.00160713273372)
result: 92 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 2, 2, 1) = fail
try: 92 - ('track_face', 'track_fa

======================= terminated 14: first answer acquired ===============================
branching: 99->100 (5.49/200.0 s, steps/err: 23(139.758825302 ms)/0.00148319173627)
======================= terminated 4: first answer acquired from other agent ===============================
transition motion tried: True
result: 87 - ('track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 2, 2, 0) = success
branching: 87->101 (5.63/200.0 s, steps/err: 199(1171.53477669 ms)/0.00170667897642)
======================= terminated 10: first answer acquired from other agent ===============================
transition motion tried: True
result: 89 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = success
branching: 89->102 (6.38/200.0 s, steps/err: 47(1862.5421524 ms)/0.00192526630742)
======================= terminated 2: first answer acquired from other agent ===============================
transition motion tried: False
result: 6 - ('grip1', 'track_face', 0, 0, 0)->

======================= terminated 13: first answer acquired from other agent ===============================
transition motion tried: False
result: 81 - ('grip1', 'track_face', 2, 0, 0)->('grip1', 'track_face', 2, 1, 0) = fail
======================= terminated 7: first answer acquired from other agent ===============================
transition motion tried: False
result: 87 - ('track_face', 'track_face', 2, 2, 0)->('track_face', 'grip1', 2, 2, 0) = fail
======================= terminated 16: first answer acquired from other agent ===============================


In [42]:
print(gtimer)

plan: 	5487.0 ms/1 = 5487.089 ms (5487.089/5487.089)



In [43]:
# for i, sidx in enumerate([1, 2, 4, 6]):
#     save_json("traj{}.json".format(i), tplan.snode_dict[sidx].traj)
#     gscene.show_motion(tplan.snode_dict[sidx].traj, period=0.05)
        

In [44]:
# import matplotlib.pyplot as plt
# kin = stereo.camera_list[0]
# rs = stereo.camera_list[1]

In [45]:
# plt.figure(figsize=(15,10))
# plt.imshow(kin.get_image())

In [46]:
# plt.figure(figsize=(15,10))
# plt.imshow(rs.get_image())

## play searched plan

In [47]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [48]:
ppline.play_schedule(snode_schedule, period=0.03)

In [49]:
# print("schedule length: {}".format(len(snode_schedule)))
# for i_s,  snode in enumerate(snode_schedule):
#     print("{}: {}".format(i_s, snode.state.node))

In [50]:
# save_json("traj.json", snode_schedule[1].traj)

## execute plan

In [51]:
crob.reset_connection(True,True)

connection command:
indy0: True
panda1: True
Connect: Server IP (192.168.0.63)


In [52]:
indy = crob.robot_dict["indy0"]
panda = crob.robot_dict["panda1"]
with indy:
    indy.set_joint_vel_level(1)
    indy.set_task_vel_level(1)

Connect: Server IP (192.168.0.63)


In [53]:
crob.joint_move_make_sure(crob.home_pose)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


In [54]:
indy.stop_tracking()

Connect: Server IP (192.168.0.63)


{'stop': True}

In [55]:
crob.grasp(indy0=False, panda1=False)

Connect: Server IP (192.168.0.63)


In [56]:
ppline.execute_schedule(snode_schedule, vel_scale=0.5, acc_scale=0.5)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
binder: track_face
rname: None
binder: track_face
rname: None
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
go
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
go
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: None
binder: None
binder: None
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip1
rname: panda1
binder: track_face
rname: None
binder: brush_face
rname: indy0
binder: None
binder: None
Connect: Server IP (192.168.0.63)
go
Connect: Serv